In [40]:
import numpy as np
import pandas as pd 
import os
from sktime.datasets import load_from_arff_to_dataframe
from pyts.utils import windowed_view
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis

In [2]:
DATA_PATH = "data"
X_train, y_train = load_from_arff_to_dataframe(
    os.path.join(DATA_PATH, "RacketSports/RacketSports_TRAIN.arff")
)

X_test, y_test = load_from_arff_to_dataframe(
    os.path.join(DATA_PATH, "RacketSports/RacketSports_TEST.arff")
)

In [3]:
def dataframe2numpy(X):
    N = len(X)
    S = len(X.iloc[0][0])
    H = len(X.columns)
    return np.stack(X.values.reshape(-1)).reshape(N,S,H)

In [4]:
rs_train = dataframe2numpy(X_train)

In [5]:
rs_train.shape

(151, 30, 6)

In [43]:
# Series mean per axis (151, 6)
rs_train.mean(1)

# Series std per axis (151, 6)
rs_train.std(1)

# Series mean absolute dev per axis (151, 6)
np.abs(rs_train - rs_train.mean(1)[:, np.newaxis, :]).mean(1)

# Series min per axis (151, 6)
rs_train.min(1)

# Series min per axis (151, 6)
rs_train.max(1)

# Series Diff max-min per axis
rs_train.max(1) - rs_train.min(1)

# Series median per axis
np.median(rs_train, axis=1)

# Series median absolute dev
np.median(np.abs(rs_train - np.median(rs_train,1)[:, np.newaxis, :]),1)

# Series IQR per axis
np.subtract(*np.percentile(rs_train,[75, 25],1))

# Series count negative per axis
np.count_nonzero(rs_train < 0, 1)

# Series count positive per axis
np.count_nonzero(rs_train >= 0, 1)

# Series count above mean per axis
np.count_nonzero(rs_train > rs_train.mean(1)[:, np.newaxis, :], 1)

# Series count peaks per axis
np.apply_along_axis(lambda x: len(find_peaks(x)[0]), 1, rs_train)

# Series skew
skew(rs_train,axis=1)

# Series kurtosis
kurtosis(rs_train, axis=1)

# Series energy per axis
np.mean(rs_train**2,axis=1)

# Series acceleration
np.mean(np.sqrt(np.sum(rs_train**2,axis=-1)),axis=1)[:, np.newaxis]

# Series signal mag area
np.sum(np.mean(np.abs(rs_train),axis=1),axis=1)[:, np.newaxis].shape

(151, 1)

In [44]:
q = np.stack([windowed_view(rs_train[:,:,idx],10,3) for idx in range(rs_train.shape[-1])], axis=-1)
q.shape

(151, 7, 10, 6)